# Summary

When providing the same input parameters and units, class and classy has the same outputs.

In [ ]:
import os
import io
from pathlib import Path

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
import plotly.offline as py
py.init_notebook_mode()

In [ ]:
import holoviews as hv
hv.extension('plotly')

In [ ]:
IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dautil.plot import iplot_column_slider, plot_column_slider

In [ ]:
def read_txt(path):
    with open(path, 'r') as f:
        # remove beginning `#`
        text = f.read()[1:]
    with io.StringIO(text) as f:
        # input has 6 sig. fig. which fits in float32
        return pd.read_csv(f, delim_whitespace=True, index_col=0, dtype=np.float32)

In [ ]:
def read_class_txt(path, camb=False):
    # read once
    with open(path, 'r') as f:
        text = f.read()

    # get last comment line
    comment = None
    for line in text.split('\n'):
        if line.startswith('#'):
            comment = line
    # remove beginning '#'
    comment = comment[1:]
    # parse comment line: get name after ':'
    names = [name.split(':')[1] for name in comment.strip().split()]

    with io.StringIO(text) as f:
        df = pd.read_csv(f, delim_whitespace=True, index_col=0, comment='#', header=None, names=names)
    df.columns.name = 'spectra'
    return df if camb else df * 1.e12

In [ ]:
def plot_compare(df1, df2, keys, log=False, relative=False):
    '''plot to compare columns from `df1` and `df2`,
    using columns from `df1`.

    `keys`: tuple of str of names of `df1` and `df2`.
    `relative`: if True, plot the relative error w.r.t. `df1` instead.
    '''
    for col in df1.columns:
#         pd.merge(df1[col], df2[col], left_index=True, right_index=True).plot()
        df_temp = pd.concat((df1[col], df2[col]), axis=1, join='inner', keys=(' '.join((key, col)) for key in keys))
        if relative:
            temp = df_temp.values
            if log:
                plt.loglog(df_temp.index, np.abs((temp[:, 0] - temp[:, 1]) / temp[:, 0]), label=col)
            else:
                plt.plot(df_temp.index, np.abs((temp[:, 0] - temp[:, 1]) / temp[:, 0]), label=col)
            plt.legend()
            plt.show()
        else:
            if log:
                df_temp.plot(logx=True, logy=True)
            else:
                df_temp.plot()

In [ ]:
# first value is default
# rest are alternative value for each realization
PARAMS = {
    'STELLAR_BARYON_FRAC': (0.05, 0.001, 1.),
    'STELLAR_BARYON_PL': (0.5, -0.5, 1.),
    'ESC_FRAC': (0.01, 0.001, 1.),
    'ESC_PL': (-0.5, -1., 0.5),
    'M_TURNOVER': (5e8, 1e8, 1e10),
    't_STAR': (0.5, 0., 1.),
    'L_X': (40.5, 38., 42.)
}

In [ ]:
def parse_case(string):
    for level, idx in enumerate(map(int, string.split('_'))):
        if idx != 0:
            break
    if idx == 0:
        return 'default'
    else:
        param = list(PARAMS)[level]
        return '{}={:.6}'.format(param, PARAMS[param][idx])

In [ ]:
basedir = Path('~/git/source/UCB-PHYS229-2019Spring-final-project/output').expanduser()

In [ ]:
df_path = pd.DataFrame(basedir.glob('*-cl_lensed.dat'), columns=['path'])

In [ ]:
df_path['case'] = df_path.path.map(lambda path: path.name.split('-')[0])

In [ ]:
df_path['name'] = df_path.case.map(parse_case)

In [ ]:
df_path

# Class result from command line

In [ ]:
df = pd.concat(
    (read_class_txt(path, camb=True) for path in df_path.path),
    axis=1,
    keys=df_path.name
#     keys=df_path.case
)

In [ ]:
df.columns.names = ('case', 'spectra')

In [ ]:
plot_column_slider(df, slider=True)

In [ ]:
for spectrum in df.columns.levels[1]:
    py.plot(iplot_column_slider(df.T.loc[IDX[:, spectrum], :].T), filename=f'../docs/media/{spectrum}.html')

In [ ]:
for spectrum in df.columns.levels[1]:
    print(f'![{spectrum}](media/{spectrum}.html){{width=100% height=500}}', end='\n\n')